In [ ]:
import datetime
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
kf_logostic = pd.read_excel("itog.xlsx")

def f(x):
    if (x['Опоздание доставка общ'] >= 0):
        return 0
    else:
        return 1
    
def f1(x):
    if (x['Опоздание кухня'] >= 0):
        return 0
    else:
        return 1
    
def f2(x):
    if (x['Опоздание кухня'] < 0):
        return x['Опоздание доставка общ'] - x['Опоздание кухня']
    else:
        return x['Опоздание доставка общ']
    
    
def f3(x):
    if (x['Опоздание только курьеры'] >= 0):
        return 0
    else:
        return 1 

def f4(x):
    if pd.isna(x['Источник заказа']):
        return 'kvikfud.ru'
    else:
        return x['Источник заказа'] 

kf_logostic['Время доставки'] = kf_logostic['Фактическое время доставки'] - kf_logostic['Время открытия']
kf_logostic['Время доставки'] = kf_logostic['Время доставки'].dt.total_seconds()/60

kf_logostic['Опоздание доставка общ'] = kf_logostic['Планируемое время доставки'] - kf_logostic['Фактическое время доставки']
kf_logostic['Опоздание доставка общ'] = kf_logostic['Опоздание доставка общ'].dt.total_seconds()/60

kf_logostic['озвучено время курьерами'] = kf_logostic['Планируемое время доставки'] - kf_logostic['Время открытия']
kf_logostic['озвучено время курьерами'] = kf_logostic['озвучено время курьерами'].dt.total_seconds()/60 - kf_logostic['время заявленное кухней']

kf_logostic['озвучено время курьерами'] = kf_logostic['озвучено время курьерами'].round(decimals=-1)

kf_logostic['Флаг опоздания общ'] = kf_logostic.apply(f, axis=1)
kf_logostic['Флаг опоздания общ'].value_counts()

print('Доставлено заказов вовремя: ', kf_logostic['Флаг опоздания общ'].value_counts()[0])
print('Доставлено заказов с опозданием: ', kf_logostic['Флаг опоздания общ'].value_counts()[1])
print('Процент опозданий заказов: ', round(100*kf_logostic['Флаг опоздания общ'].value_counts()[1]/(kf_logostic['Флаг опоздания общ'].value_counts()[1]+kf_logostic['Флаг опоздания общ'].value_counts()[0]),2),'%')
print('\n')

kf_logostic['Время приготовления'] = kf_logostic['Время окончания приготовления'] - kf_logostic['Время открытия']
kf_logostic['Время приготовления'] = kf_logostic['Время приготовления'].dt.total_seconds()/60

kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['Тип заказа'] == 'На время самовывоз'].index)
kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['Тип заказа'] == 'На время курьером'].index)

kf_logostic['Опоздание кухня'] = kf_logostic['время заявленное кухней']- kf_logostic['Время приготовления']

kf_logostic['Флаг опоздания кухня'] = kf_logostic.apply(f1, axis=1)
kf_logostic['Флаг опоздания кухня'].value_counts()

print('Приготовлено вовремя заказов: ', kf_logostic['Флаг опоздания кухня'].value_counts()[0])
print('Опозданий в приготовлении: ', kf_logostic['Флаг опоздания кухня'].value_counts()[1])
print('Процент опозданий в приготовлении: ', round(100*kf_logostic['Флаг опоздания кухня'].value_counts()[1]/(kf_logostic['Флаг опоздания кухня'].value_counts()[1]+kf_logostic['Флаг опоздания кухня'].value_counts()[0]),2),'%')
print('\n')

kf_logostic_without_bad_closed = kf_logostic.drop(kf_logostic[kf_logostic['заказы не для учета'] == 1].index)
kf_logostic_without_bad_closed = kf_logostic_without_bad_closed.drop(kf_logostic_without_bad_closed[kf_logostic_without_bad_closed['Тип заказа'] == 'Доставка самовывоз'].index)
kf_logostic_without_bad_closed['Опоздание только курьеры'] = kf_logostic_without_bad_closed.apply(f2, axis=1)
kf_logostic_without_bad_closed['Флаг опоздание только курьеры'] = kf_logostic_without_bad_closed.apply(f3, axis=1)

print('Доставлено курьерами вовремя: ', kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[0])
print('Опозданий по вине курьеров: ', kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1])
print('Процент опозданий по вине курьеров: ', round(100*kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1]/(kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1]+kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[0]),2),'%')
print('\n')

#kf_logostic_without_bad_closed['время заявленное кухней'].value_counts()
kf_logostic_without_bad_closed['Источник заказа'] = kf_logostic_without_bad_closed.apply(f4, axis=1)
kf_logostic_without_delivery = kf_logostic_without_bad_closed.drop(kf_logostic_without_bad_closed[kf_logostic_without_bad_closed['Источник заказа'] == 'delivery-club.ru'].index)
print(kf_logostic_without_delivery.groupby('озвучено время курьерами')['Источник заказа'].value_counts())

kf_logostic_without_delivery.loc[(kf_logostic_without_delivery['озвучено время курьерами']>70) | (kf_logostic_without_delivery['озвучено время курьерами']<20)].head(10)

otchet=kf_logostic_without_delivery.loc[(kf_logostic_without_delivery['озвучено время курьерами']>70) | (kf_logostic_without_delivery['озвучено время курьерами']<20)]
otchet.to_excel('проблемные заказы.xlsx')